In [20]:
cd ..

SyntaxError: invalid syntax (3729187224.py, line 1)

In [8]:
import hexagon
import utils
import h3.api.numpy_int as h3
import numpy as np

In [2]:
df = utils.read_parquet("Taxi_Trips_cleaned.parquet", columns=[
    "Pickup Centroid Latitude",
    "Pickup Centroid Longitude",
    "Dropoff Centroid Latitude",
    "Dropoff Centroid Longitude"
])

In [3]:
#df = df.sample(1000)

## Converting coordinates to hexagons
Hexagons are represented as h3 indexes in h3. Conversion does not allow vectorization by default, but there is an
experimental version: https://github.com/uber/h3-py/issues/141
Vectoring (hexagon.get_hexagon_vect()) only takes 16 seconds compared to 4 minutes without vectorization.

Results can be seen below.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22026534 entries, 0 to 22026533
Data columns (total 4 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Pickup Centroid Latitude    float64
 1   Pickup Centroid Longitude   float64
 2   Dropoff Centroid Latitude   float64
 3   Dropoff Centroid Longitude  float64
dtypes: float64(4)
memory usage: 672.2 MB


In [5]:
%%time
df["hex_vect"] = hexagon.get_hexagon_vect(df["Pickup Centroid Latitude"], df["Pickup Centroid Longitude"], res=8)

CPU times: user 16.1 s, sys: 274 ms, total: 16.4 s
Wall time: 16.5 s


In [6]:
%%time
df["hex_normal"] =  df.apply(lambda x: h3.geo_to_h3(x["Pickup Centroid Latitude"], x["Pickup Centroid Longitude"], resolution=8), axis=1 )

CPU times: user 4min 13s, sys: 2.99 s, total: 4min 16s
Wall time: 4min 16s


In [10]:
df["compare"] = np.where(df["hex_vect"] == df["hex_normal"], True, False)
df['compare'].value_counts()

## Creating GeoJson with boundries of hexagons

